In [25]:
from urllib.request import urlopen

from serpapi import GoogleSearch
# pip3 install google-search-results

import os

import sys

import pprint

import pandas as pd

import csv

In [26]:
def google_search(query, page, paging):
    params = {
      "engine": "google_scholar",
      "hl": "en",
      "q": query, # query string
      "start": page, # starting page offset (0 first page -> 10 second page -> 20 third page)
      "num": paging, # results per page (max = 20),
      "as_vis": 1, # 1: exclude citations, 0: include citations
      "api_key": "XXX" # Google API key
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    if (results.get('search_information', None) != None) and (results.get('organic_results', None) != None):
        search_information = results['search_information']
        organic_results = results['organic_results']
        # related_results = results['related_searches'] # not needed
        return search_information, organic_results

In [49]:
# QUERY
query_num = 6 # (following the query_x variable number)
query_1 = "public procurement machine learning"
query_2 = "public procurement deep learning"
query_3 = "public procurement recourse prediction"
query_4 = "legal document recommendation system"
query_5 = "automatic legal research recommendation"
query_6 = "legal research recommendation" 
if query_num == 1:
    query = query_1
if query_num == 2:
    query = query_2
if query_num == 3:
    query = query_3
if query_num == 4:
    query = query_4
if query_num == 5:
    query = query_5
if query_num == 6:
    query = query_6

In [50]:
# Results per page
paging = 10 # results per page (max = 20)

In [51]:
# First call to collect the result number
# search_information, organic_results = google_search(query, 0, paging)
# total_results = search_information['total_results']
# print(total_results)

In [52]:
# Get the total pages
# pages = int(round(total_results / paging, 0))
# pages

In [53]:
final_df = pd.DataFrame() # container of all the results

In [54]:
dim = 10 # number of pages (-> dim * paging results)
for i in range (0,dim):
    start = i * paging # 0 * 10 -> 0 first page, 1 * 10 -> 10 (second page), 2 * 10 -> 20 (third page), 100 last page
    search_information, organic_results = google_search(query, start, paging)
    for data in organic_results:
        # print(data) # debug
        line_df = pd.json_normalize(data) # transforms a result line into a dataframe
        # line_df = line_df.set_index("position")
        # df.head()
        # print(line_df.shape) # debug
        final_df = pd.concat([line_df, final_df], ignore_index=True)

In [55]:
final_df.shape

(100, 22)

In [56]:
final_df = final_df.sort_values(by = ['inline_links.cited_by.total'], ascending = False) # sort by citations 
# col_remove = ['inline_links.versions.cluster_id', 'inline_links.serpapi_cite_link', 'inline_links.related_pages_link', 'inline_links.serpapi_related_pages_link']
# final_df.drop(col_remove)
final_df = final_df.drop_duplicates(subset=['result_id'])

In [57]:
final_df.head()

,position,title,result_id,link,snippet,resources,publication_info.summary,publication_info.authors,inline_links.serpapi_cite_link,inline_links.cited_by.total,...,inline_links.cited_by.serpapi_scholar_link,inline_links.related_pages_link,inline_links.serpapi_related_pages_link,inline_links.versions.total,inline_links.versions.link,inline_links.versions.cluster_id,inline_links.versions.serpapi_scholar_link,type,inline_links.html_version,inline_links.cached_page_link
43,6,The health effects of cannabis and cannabinoid...,r165UZai7XsJ,https://books.google.com/books?hl=en&lr=&id=FT...,"… , conflicting and impeded scientific researc...","[{'title': 'nih.gov', 'file_format': 'HTML', '...","National Academies of Sciences, Engineering, a...",NaN,https://serpapi.com/search.json?engine=google_...,1502.0,...,https://serpapi.com/search.json?as_sdt=8000000...,https://scholar.google.com/scholar?q=related:r...,https://serpapi.com/search.json?as_sdt=0%2C23&...,9.0,https://scholar.google.com/scholar?cluster=892...,8929972402645458607,https://serpapi.com/search.json?as_sdt=0%2C23&...,NaN,https://www.ncbi.nlm.nih.gov/sites/books/n/nap...,NaN
18,1,Eyewitness identification procedures: Recommen...,yMRSmk8ORBcJ,https://psycnet.apa.org/doiLanding?doi=10.1023...,… that law enforcement investigators must foll...,"[{'title': 'apa.org', 'file_format': 'HTML', '...","GL Wells, M Small, S Penrod, RS Malpass… - Law...","[{'name': 'GL Wells', 'link': 'https://scholar...",https://serpapi.com/search.json?engine=google_...,1381.0,...,https://serpapi.com/search.json?as_sdt=2005&ci...,https://scholar.google.com/scholar?q=related:y...,https://serpapi.com/search.json?as_sdt=0%2C5&a...,19.0,https://scholar.google.com/scholar?cluster=167...,1676480696342987976,https://serpapi.com/search.json?as_sdt=0%2C5&a...,Html,https://psycnet.apa.org/doiLanding?doi=10.1023...,NaN
95,4,Towards a standard for pointing device evaluat...,WaMvGuI0O7wJ,https://www.sciencedirect.com/science/article/...,"… In this paper, we will make seven recommenda...","[{'title': 'yorku.ca', 'file_format': 'HTML', ...","RW Soukoreff, IS MacKenzie - International jou...","[{'name': 'IS MacKenzie', 'link': 'https://sch...",https://serpapi.com/search.json?engine=google_...,1068.0,...,https://serpapi.com/search.json?as_sdt=5%2C33&...,https://scholar.google.com/scholar?q=related:W...,https://serpapi.com/search.json?as_sdt=0%2C33&...,8.0,https://scholar.google.com/scholar?cluster=135...,13563492848462963545,https://serpapi.com/search.json?as_sdt=0%2C33&...,NaN,http://www.yorku.ca/mack/ijhcs2004.html,NaN
94,5,Managing incidental findings in human subjects...,o54Y_YGlF2IJ,https://www.cambridge.org/core/journals/journa...,"… Alternatively, researchers following our rec...","[{'title': 'wiley.com', 'file_format': 'PDF', ...","SM Wolf, FP Lawrenz, CA Nelson, JP Kahn… - Jou...","[{'name': 'CA Nelson', 'link': 'https://schola...",https://serpapi.com/search.json?engine=google_...,803.0,...,https://serpapi.com/search.json?as_sdt=5%2C33&...,https://scholar.google.com/scholar?q=related:o...,https://serpapi.com/search.json?as_sdt=0%2C33&...,28.0,https://scholar.google.com/scholar?cluster=706...,7068300117896896163,https://serpapi.com/search.json?as_sdt=0%2C33&...,NaN,NaN,NaN
88,1,Defining and describing what we do: doctrinal ...,_JLk21JdWK0J,https://search.informit.org/doi/pdf/10.3316/ag...,… used in legal research and its place in rece...,"[{'title': 'city.ac.uk', 'file_format': 'PDF',...","T Hutchinson, N Duncan - Deakin Law Review, 20...",NaN,https://serpapi.com/search.json?engine=google_...,662.0,...,https://serpapi.com/search.json?as_sdt=2005&ci...,https://scholar.google.com/scholar?q=related:_...,https://serpapi.com/search.json?as_sdt=0%2C5&a...,15.0,https://scholar.google.com/scholar?cluster=124...,12490836176969962236,https://serpapi.com/search.json?as_sdt=0%2C5&a...,NaN,NaN,NaN


In [58]:
final_df.shape

(99, 22)

In [59]:
file_name = "gs_results_query_" +str(query_num)+".csv"
final_df.to_csv(file_name, index = False, header = True, sep = ";", quoting = csv.QUOTE_ALL)

In [187]:
# Merging all results
merge_df = pd.DataFrame() # container of all the results
for i in range(1,query_num+1):
    file_name = "gs_results_query_" +str(i)+".csv"
    print(file_name)
    temp_df = pd.read_csv(file_name, sep = ";")
    temp_df = temp_df.drop_duplicates(subset=['result_id'])
    print("Temp df shape:", temp_df.shape)
    merge_df = pd.concat([merge_df, temp_df])
    print("Merge df shape:",merge_df.shape)

gs_results_query_1.csv
Temp df shape: (99, 22)
Merge df shape: (99, 22)
gs_results_query_2.csv
Temp df shape: (99, 22)
Merge df shape: (198, 22)
gs_results_query_3.csv
Temp df shape: (100, 22)
Merge df shape: (298, 22)


In [188]:
merge_df = merge_df.drop_duplicates(subset=['result_id'])
merge_df.shape

(104, 22)

In [189]:
file_name = "gs_results_query_all.csv"
final_df.to_csv(file_name, index = False, header = True, sep = ";", decimal=',', quoting = csv.QUOTE_ALL)
# decimal=',' for italian culture